In [136]:
!pip install geopandas 

In [24]:
# 필요 라이브러리 임포트
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt
import pandas as pd

import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

# 1. 특별시 시군구 거리계산

In [14]:
everything_add_df = pd.read_csv("../data/everything_add.csv", index_col=0)
address_df = pd.read_csv("../data/address.csv", index_col=0)

In [23]:
everything_add_df

,name,address,Coordination,위도,경도,geometry
0,서울특별시,(04524) 서울특별시 중구 세종대로 110 서울특별시청,"(37.566370748, 126.977918341)",37.566371,126.977918,POINT (37.566370748 126.977918341)
1,부산광역시,(47545) 부산광역시 연제구 중앙대로 1001(연산동) 부산광역시청,"(35.179819016, 129.075092136)",35.179819,129.075092,POINT (35.179819016 129.075092136)
2,대구광역시,(41911) 대구광역시 중구 공평로 88 (동인동1가) 대구광역시청,"(35.871565392, 128.601266169)",35.871565,128.601266,POINT (35.871565392 128.601266169)
3,인천광역시,(21554) 인천광역시 남동구 정각로29 (구월동 1138) 인천광역시청,"(37.456317223, 126.704380115)",37.456317,126.704380,POINT (37.456317223 126.704380115)
4,광주광역시,(61945) 광주광역시 서구 내방로 111(치평동) 광주광역시청,"(35.16012861, 126.851451621)",35.160129,126.851452,POINT (35.16012861 126.851451621)
5,대전광역시,(35242) 대전 서구 둔산로 100 대전광역시청,"(36.3502901, 127.384984276)",36.350290,127.384984,POINT (36.3502901 127.384984276)
6,울산광역시,(44675) 울산광역시 남구 중앙로 201 (신정동) 울산광역시청,"(35.539590906, 129.311548261)",35.539591,129.311548,POINT (35.539590906 129.311548261)
7,세종특별자치시,(30151) 세종특별자치시 한누리대로 2130 (보람동) 세종특별자치시청,"(36.480112199, 127.289097137)",36.480112,127.289097,POINT (36.480112199 127.289097137)
8,경기도,(16508) 경기도 수원시 영통구 도청로 30 경기도청,"(37.289285272, 127.053543354)",37.289285,127.053543,POINT (37.289285272 127.053543354)
9,강원특별자치도,(24266) 강원도 춘천시 중앙로1 (봉의동) 강원도청,"(37.885424118, 127.729638401)",37.885424,127.729638,POINT (37.885424118 127.729638401)


In [15]:
address_df

,name,address,Coordination
0,서울 종로구,종로구 삼봉로 43,"(37.572984387, 126.979707283)"
1,서울 중구,중구 창경궁로 17,"(37.563796112, 126.997564683)"
2,서울 용산구,용산구 녹사평대로 150,"(37.53264173, 126.990444235)"
3,서울 성동구,성동구 고산자로 270,"(37.563428014, 127.036908072)"
4,서울 광진구,광진구 자양로 117,"(37.537914041, 127.082219934)"
...,...,...,...
258,거 창 군,거창군 거창읍 중앙로 103,"(35.686682309, 127.909545955)"
259,합 천 군,합천군 합천읍 동서로 119,"(35.566655186, 128.165830822)"
260,제 주 시,제주시 광양9길 10,"(33.499544449, 126.531113201)"
261,서귀포시,서귀포시 중앙로 105 (제1청사),"(33.253953653, 126.55958856)"


In [41]:
everything_add_df['geometry'] = everything_add_df['Coordination'].apply(lambda x: wkt.loads('POINT' + x.replace(',', '')))

In [27]:
address_df['geometry'] = address_df['Coordination'].apply(lambda x: wkt.loads('POINT' + x.replace(',', '')))

In [46]:
# GeoDataFrame으로 변환
everything_add_df_gdf = gpd.GeoDataFrame(everything_add_df, geometry='geometry')
address_df_gdf = gpd.GeoDataFrame(address_df, geometry='geometry')

In [52]:
# 좌표계 설정 (WGS 84)
gdf = gdf.set_crs(epsg=4326)

# 결과를 저장할 DataFrame
results = pd.DataFrame(columns=['특별시', '50km이내_시군구_개수', '100km이내_시군구_개수', '가장_가까운_시군구', '거리(km)'])

for idx, city in everything_add_df_gdf.iterrows():
    # 특별시와 모든 시군구 간의 거리 계산
    distances = address_df_gdf.geometry.distance(city.geometry) * 100  # 거리를 km 단위로 변환
    near_50km = distances[distances <= 50].count()
    near_100km = distances[distances <= 100].count()
    nearest_district_idx = distances.idxmin()
    nearest_distance = distances.min()
    
    # 결과 저장
    results = results.append({
        '특별시': city['name'],
        '50km이내_시군구_개수': near_50km,
        '100km이내_시군구_개수': near_100km,
        '가장_가까운_시군구': address_df_gdf.loc[nearest_district_idx, 'name'],
        '거리(km)': nearest_distance
    }, ignore_index=True)

results

,특별시,50km이내_시군구_개수,100km이내_시군구_개수,가장_가까운_시군구,거리(km)
0,서울특별시,77,100,서울 종로구,0.685132
1,부산광역시,29,54,부산 연제구,0.500734
2,대구광역시,19,72,대구 중구,0.540591
3,인천광역시,71,96,인천 남동구,2.898165
4,광주광역시,19,45,광주 서구,3.967140
5,대전광역시,21,65,대전 서구,0.531360
6,울산광역시,20,50,울산 남구,1.898548
7,세종특별자치시,25,78,대전 유성구,13.562708
8,경기도,81,113,수원시 영통구,3.049015
9,강원특별자치도,9,81,춘 천 시,0.503485


In [92]:
#결과물 저장
results.to_csv('result.csv', encoding='utf-8-sig', index=False)